In [10]:
import requests
import json
import time
import pandas as pd
import numpy as np

In [17]:
URL = "http://api.weatherapi.com/v1/history.json?key=<YOUR_API_KEY>&q={}&dt={}"

STATES_CITIES = {'Andhra Pradesh' : ["Amaravati",	"Visakhapatnam", "Vijayawada", "Guntur", "Nellore", "Tirupati"],
                 'Arunachal Pradesh' : ["Itanagar", "Tawang", "Bhismaknagar", "Pasighat", "Ziro", "Bomdila"],
                 'Assam' : ["Dispur", "Guwahati", "Tezpur", "Dibrugarh", "Silchar", "North Lakhimpur"],
                 'Bihar' : ["Patna", "Gaya", "Biharsharif", "Darbhanga", "Bhagalpur"],
                 'Chhattisgarh' : ["Raipur", "Bilaspur", "Korba", "Raigarh", "Rajnandgaon"],
                 'Goa' : ["Panaji", "Vasco-da-Gama", "Ponda", "Margao", "Mapusa", "Goa Velha"],
                 'Gujarat' : ["Gandhinagar", "Ahmedabad", "Surat", "Rajkot", "Junagadh", "Vadodara"],
                 'Haryana' : ["Chandigarh", "Faridabad", "Gurgaon", "Sonipat", "Panipat", "Ambala"],
                 'Himachal Pradesh' : ["Shimla", "Dharamshala", "Mandi", "Solan", "Bilaspur", "Chamba"],
                 'Jharkhand' : ["Ranchi", "Bokaro Steel City", "Jamshedpur", "Deoghar", "Hazaribagh", "Dhanbad"],
                 'Karnataka' : ["Bengaluru", "Mysore", "Davangere", "Mangalore", "Hubli-Dharwad", "Belgaum"],
                 'Kerala' : ["Thiruvananthapuram", "Kochi", "Kozhikode", "Thrissur", "Malappuram"],
                 'Madhya Pradesh' : ["Bhopal", "Indore", "Gwalior", "Jabalpur", "Ujjain", "Sagar"],
                 'Maharashtra' : ["Mumbai", "Pune", "Nagpur", "Nashik", "Aurangabad", "Solapur"],
                 'Manipur' : ["Imphal", "Bishnupur", "Ukhrul", "Tamenglong", "Chandel", "Senapati"],
                 'Meghalaya' : ["Shillong", "Cherrapunji", "Tura", "Jowai", "Baghmara", "Nongpoh"],
                 'Mizoram' : ["Aizawl", "Lunglei", "Serchhip", "Champhai", "Tuipang", "Mamit"],
                 'Nagaland' : ["Kohima", "Tuensang", "Zunheboto", "Mokokchung", "Kiphire Sadar", "Phek"],
                 'Odisha' : ["Bhubaneswar", "Rourkela", "Cuttack", "Brahmapur", "Puri", "Sambalpur"],
                 'Punjab' : ["Chandigarh", "Amritsar", "Jalandhar", "Ludhiana", "Patiala", "Kapurthala"],
                 'Rajasthan' : ["Jaipur", "Bikaner", "Jaisalmer", "Jodhpur", "Udaipur", "Ajmer"],
                 'Sikkim' : ["Gangtok", "Namchi", "Gyalshing", "Mangan", "Rabdentse"],
                 'Tamil Nadu' : ["Chennai", "Tiruchirappalli", "Madurai", "Erode", "Vellore", "Coimbatore"],
                 'Telangana' : ["Hyderabad", "Warangal", "Nizamabad", "Karimnagar", "Adilabad", "Khammam"],
                 'Tripura' : ["Agartala", "Amarpur", "Kumarghat", "Udaipur", "Gakulnagar", "Kunjaban"],
                 'Uttar Pradesh' : ["Lucknow", "Noida", "Varanasi", "Allahabad", "Agra", "Kanpur"],
                 'Uttarakhand' : ["Dehradun", "Haridwar", "Roorkee", "Rishikesh", "Kashipur", "Haldwani"],
                 'West Bengal' : ["Kolkata", "Darjeeling", "Siliguri", "Asansol", "Howrah", "Durgapur"]
                 }

date_format = ["2021-10-{}".format(i) for i in range(15, 23)]

def weather_API(STATES_CITIES, URL):

  # Dataframe to store location
  location_df = pd.DataFrame(columns = ['name', 'region', 'country', 'lat', 'lon', 'tz_id', 'localtime_epoch', 'localtime'])
  # Dataframe to store forecasts
  forecast_df = pd.DataFrame(columns = ['time_epoch', 'time', 'temp_c', 'temp_f', 'is_day', 'condition',
                                        'wind_mph', 'wind_kph', 'wind_degree', 'wind_dir', 'pressure_mb',
                                        'pressure_in', 'precip_mm', 'precip_in', 'humidity', 'cloud',
                                        'feelslike_c', 'feelslike_f', 'windchill_c', 'windchill_f',
                                        'heatindex_c', 'heatindex_f', 'dewpoint_c', 'dewpoint_f',
                                        'will_it_rain', 'chance_of_rain', 'will_it_snow', 'chance_of_snow',
                                        'vis_km', 'vis_miles', 'gust_mph', 'gust_kph', 'state','city'])
  # Dataframe to store astro
  astro_df = pd.DataFrame(columns = ['sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination', 'state', 'city'])
    
  # iterate over states
  for state in STATES_CITIES.keys():
    print("Extracting data for state {}".format(state))
    print()

    # iterate over all cities
    cities = STATES_CITIES[state]
    state_arr = np.array([state for _ in range(24)])

    for j, date in enumerate(date_format):

        print('Date : {}'.format(date))
        city_count = 0

        for i, city in enumerate(cities):

          print("Extracting data for city {}".format(city))
          print()

          # Create the final URL
          final_url = URL.format(city, date)

          # make get request
          try:
            time.sleep(10)
            page_response = requests.get(final_url)
          except:
            print("Could not make GET request for state : {} city : {} and date : {}".format(city, state, date))

          # extract the page_response content
          data_dict = eval(page_response.content)


          try:
            # location_data
            location_data = data_dict['location']
            curr_loc_df = pd.DataFrame(data = location_data, index = [i])

            # concat with main frame
            location_df = pd.concat((location_df, curr_loc_df), axis=0)
          except:

            print("Could not extract location data for state : {} city : {} date: {}".format(state, city, date))

          try:
            # forecast_data
            forecast_data = data_dict['forecast']['forecastday'][0]['hour']
            curr_forecast_df = pd.DataFrame(forecast_data)
            num_rows = curr_forecast_df.shape[0]

            # Create city arr
            city_arr = np.array([city for _ in range(num_rows)])

            # Add state array and city array in current forecast df
            curr_forecast_df['city'] = city_arr
            curr_forecast_df['state'] = state_arr

            # concat with main frame
            forecast_df = pd.concat((forecast_df, curr_forecast_df), axis=0)
          except:
            print("Could not extract forecast data for state : {} city : {} data : {}".format(state, city, date))
          
          try:
            # Astro data
            astro_data = data_dict['forecast']['forecastday'][0]['astro']
            curr_astro_df = pd.DataFrame(astro_data, index=[id])
            curr_astro_df['city'] = [city]
            curr_astro_df['state'] = [state]

            # Concat with main frame
            astro_df = pd.concat((astro_df, curr_astro_df), axis=0)
          except:
            print("Could not extract astro data for state : {} city : {} date : {}".format(state, city, date))
            city_count -= 1
          
            city_count += 1
        print("Extracted {} out of {} cities for state {}".format(city_count, len(cities), state))
        print()

  return location_df, forecast_df, astro_df

In [ ]:
# Extract Data
location_df, forecast_df, astro_df = weather_API(STATES_CITIES=STATES_CITIES, URL=URL)

In [24]:
forecast_df.to_csv("./forecast_data.csv", index=False)
location_df.to_csv("./location_data.csv", index=False)
astro_df.to_csv("./astro_data.csv", index=False)